In [1]:
import os
os.environ["MANTLE_TARGET"] = "ice40"
import magma as m
from magma.bitutils import int2seq

We will import a `FullAdder` implementation from `mantle` to implement an N-bit adder.

To do so, we begin by instantiating a list of N `FullAdder` instances.

We use the magma `m.fold` function to wire the carry out of each instance to the carry in of the next instance. More information on `m.fold` and other higher order circuit operators can be found [here](https://github.com/phanrahan/magma/wiki/Higher-Order-Circuit-Operators#fold-and-scan).

![](./images/fold.jpg)

In [2]:
from mantle.lattice.mantle40.fulladder import FullAdder

def DefineAdder(N):
    T = m.UInt(N)
    class Adder(m.Circuit):
        name = "Adder{}".format(N)
        IO = ["a", m.In(T), "b", m.In(T), "cin", m.In(m.Bit), "out", m.Out(T), "cout", m.Out(m.Bit)]
        @classmethod
        def definition(io):
            adders = [FullAdder() for _ in range(N)]
            circ = m.fold(adders, foldargs={"CIN":"COUT"})
            m.wire(io.a, circ.I0)
            m.wire(io.b, circ.I1)
            m.wire(io.cin, circ.CIN)
            m.wire(io.cout, circ.COUT)
            m.wire(io.out, circ.O)
    return Adder

Adder4 = DefineAdder(4)

import mantle lattice ice40
import mantle lattice mantle40


We can use magma's Python simulator to generate test vectors for our Adder circuit.

In [3]:
from magma.simulator.python_simulator import testvectors

tests = testvectors(Adder4) 
print(" a  b  ci o  co")
for test in tests:
    for t in test:
        print("{:2d}".format(t), end=' ')
    print()

 a  b  ci o  co
 0  0  0  0  0 
 0  0  1  1  0 
 0  1  0  1  0 
 0  1  1  2  0 
 0  2  0  2  0 
 0  2  1  3  0 
 0  3  0  3  0 
 0  3  1  4  0 
 0  4  0  4  0 
 0  4  1  5  0 
 0  5  0  5  0 
 0  5  1  6  0 
 0  6  0  6  0 
 0  6  1  7  0 
 0  7  0  7  0 
 0  7  1  8  0 
 0  8  0  8  0 
 0  8  1  9  0 
 0  9  0  9  0 
 0  9  1 10  0 
 0 10  0 10  0 
 0 10  1 11  0 
 0 11  0 11  0 
 0 11  1 12  0 
 0 12  0 12  0 
 0 12  1 13  0 
 0 13  0 13  0 
 0 13  1 14  0 
 0 14  0 14  0 
 0 14  1 15  0 
 0 15  0 15  0 
 0 15  1  0  1 
 1  0  0  1  0 
 1  0  1  2  0 
 1  1  0  2  0 
 1  1  1  3  0 
 1  2  0  3  0 
 1  2  1  4  0 
 1  3  0  4  0 
 1  3  1  5  0 
 1  4  0  5  0 
 1  4  1  6  0 
 1  5  0  6  0 
 1  5  1  7  0 
 1  6  0  7  0 
 1  6  1  8  0 
 1  7  0  8  0 
 1  7  1  9  0 
 1  8  0  9  0 
 1  8  1 10  0 
 1  9  0 10  0 
 1  9  1 11  0 
 1 10  0 11  0 
 1 10  1 12  0 
 1 11  0 12  0 
 1 11  1 13  0 
 1 12  0 13  0 
 1 12  1 14  0 
 1 13  0 14  0 
 1 13  1 15  0 
 1 14  0 15  0 
 1 14  1

We can also compile our circuit to verilog and inspect the output

In [4]:
m.compile("build/Adder4", Adder4, output="verilog")
with open("build/Adder4.v", "r") as adder4_verilog:
    print(adder4_verilog.read())

compiling FullAdder
compiling Adder4
module FullAdder (input  I0, input  I1, input  CIN, output  O, output  COUT);
wire  inst0_O;
wire  inst1_CO;
SB_LUT4 #(.LUT_INIT(16'h9696)) inst0 (.I0(I0), .I1(I1), .I2(CIN), .I3(1'b0), .O(inst0_O));
SB_CARRY inst1 (.I0(I0), .I1(I1), .CI(CIN), .CO(inst1_CO));
assign O = inst0_O;
assign COUT = inst1_CO;
endmodule

module Adder4 (input [3:0] a, input [3:0] b, input  cin, output [3:0] out, output  cout);
wire  inst0_O;
wire  inst0_COUT;
wire  inst1_O;
wire  inst1_COUT;
wire  inst2_O;
wire  inst2_COUT;
wire  inst3_O;
wire  inst3_COUT;
FullAdder inst0 (.I0(a[0]), .I1(b[0]), .CIN(cin), .O(inst0_O), .COUT(inst0_COUT));
FullAdder inst1 (.I0(a[1]), .I1(b[1]), .CIN(inst0_COUT), .O(inst1_O), .COUT(inst1_COUT));
FullAdder inst2 (.I0(a[2]), .I1(b[2]), .CIN(inst1_COUT), .O(inst2_O), .COUT(inst2_COUT));
FullAdder inst3 (.I0(a[3]), .I1(b[3]), .CIN(inst2_COUT), .O(inst3_O), .COUT(inst3_COUT));
assign out = {inst3_O,inst2_O,inst1_O,inst0_O};
assign cout = inst3_COUT;

Finally, we can test our circuit on the icestick by defining a `main` function. Change the values to `adder4.a` and `adder4.b`, reflash the icestick, and check the LEDs to see the output change.

In [5]:
from loam.boards.icestick import IceStick
icestick = IceStick()

icestick.Clock.on()
icestick.D1.on()
icestick.D2.on()
icestick.D3.on()
icestick.D4.on()

main = icestick.main()
adder4 = Adder4()
m.wire(m.bits(3, 4), adder4.a)
m.wire(m.bits(4, 4), adder4.b)
m.wire(0, adder4.cin)
m.wire(adder4.out, m.bits([main.D1, main.D2, main.D3, main.D4]))
m.compile("build/ice_adder", main)

compiling FullAdder
compiling Adder4
compiling main


In [6]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif ice_adder.blif' ice_adder.v
arachne-pnr -q -d 1k -o ice_adder.txt -p ice_adder.pcf ice_adder.blif
icepack ice_adder.txt ice_adder.bin

iceprog ice_adder.bin

init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x22 0x00 0x20 0x00 0x23 0x06 0x06 0x15 0xC9 0x77
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.
